In [ ]:

# kode som finner utliggere i london weather filen
import json
import numpy as np

# Åpne og les JSON-filen manuelt
with open('C:\anvendt_prog\Anvendt_prosjekt\data\london_weather.json', 'r') as file:
    data = json.load(file)

# Ekstraher relevant data (for eksempel fra en liste med dictionaries)
sample_data_with_outliers = np.array([entry['value'] for entry in data])

# Beregn gjennomsnitt og standardavvik
mean = np.mean(sample_data_with_outliers)
std_dev = np.std(sample_data_with_outliers)

# Definer grenseverdier
threshold = 2
lower_limit = mean - threshold * std_dev
upper_limit = mean + threshold * std_dev

# Finn uteliggere ved å bruke NumPy
outliers = sample_data_with_outliers[(sample_data_with_outliers < lower_limit) | (sample_data_with_outliers > upper_limit)]

# Skriv ut uteliggerne
print("Outliers:", outliers)
